In [1]:
import torch

from diffusers import DiffusionPipeline

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print('The GPU is not used.')


auth_token = ''#the token should be added here

def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=auth_token,
).to(device)

NVIDIA GeForce RTX 3090


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

c:\Users\scata\anaconda3\envs\env_diffusers\lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
from PIL import Image
import PIL.Image

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return new_im # Return the new square image

main_path = 'D:/'

# Set the path for the results folder
res_folder = main_path + 'Experiments/Diffusion/'
main_path += 'Datasets/with_masks/'

image = PIL.Image.open(main_path + 'frame_00048.png')

# Make the image square with the specified function
new_image = make_square(image)
new_image.save('D:/redim.png')

In [ ]:
from sys import platform
import PIL.Image
import os

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to get PNG files from a folder that do not have corresponding mask files
def get_pngs_without_masks(f):
    png_names = get_all_png_from_folder(f)
    return [x for x in png_names if not x.endswith('mask.png')]

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return new_im # Return the new square image

# Function to resize an image to 512x512 pixels
def resize_to_512(img):
    w, h = img.size
    if w != 512 or h != 512:
        return img.resize((512, 512), PIL.Image.Resampling.LANCZOS)
    return img

# Function to read an image, its mask, and a masked colored image from given folders
def read_one_file(full_path_separate_folder, full_path_colored_folder, current_folder, fn):
    mask_name = fn[:-4] + '_mask.png'
    image = PIL.Image.open(full_path_separate_folder + current_folder + '/' + fn)
    image = resize_to_512(image)

    mask_image = PIL.Image.open(full_path_separate_folder + current_folder + '/' + mask_name)
    mask_image = resize_to_512(mask_image)

    masked_colored_image = PIL.Image.open(full_path_colored_folder + current_folder + '/' + mask_name)
    masked_colored_image = resize_to_512(masked_colored_image)

    return image, mask_image, masked_colored_image

# Function to get paths of the separate and masked folders, and list of folders within the main path
def get_paths(main_path):
    separate_folder = 'Separate_files/'
    masked_folder = 'Mask_on_image/'

    full_path_separate_folder = main_path + separate_folder
    full_path_colored_folder = main_path + masked_folder

    return full_path_separate_folder, full_path_colored_folder, os.listdir(full_path_separate_folder)

# Function to get PNG files from a folder that do not have corresponding mask files
def get_png_without_masks(path_to_folder):
    all_files_in_folder = os.listdir(path_to_folder)
    return [x for x in all_files_in_folder if 'mask' not in x and x.endswith('.png')]

# Function to get results from a pipeline (inpainting)
def get_pipe_results(image, prompt = "background", guidance_scale=7.5, num_samples = 10):
    generator = torch.Generator(device="cuda").manual_seed(0) # change the seed to get different results
    
    import time

    start = time.time()

    pipe.safety_checker = lambda images, clip_input: (images, False)
    images = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask_image,
        guidance_scale=guidance_scale,
        generator=generator,
        num_images_per_prompt=num_samples,
    ).images
    
    return images, time.time() - start # Return the generated images and the time taken

main_path = 'D:/'

# Set the path for the results folder
res_folder = main_path + 'Experiments/Diffusion-all/'
main_path += 'Datasets/without_masks_all_reduced512_uncropped/'

# Create the results folder if it does not exist
if not os.path.exists(res_folder):
   os.makedirs(res_folder)

# Loop through each image without a mask in the main path
for image_name in get_pngs_without_masks(main_path):
    image = PIL.Image.open(main_path + image_name)

    # Save each generated image
    mask_image = PIL.Image.open(main_path + image_name[:-4] + '_mask.png')
    resulted_images, seconds = get_pipe_results(image)
    for counter, res_img in enumerate(resulted_images):
        fn_res = image_name[:-4] + '__v_' + str(counter) + '__sec_' + '{:.2f}'.format(seconds / 10) + '.png'
        res_img.save(res_folder + '/' + fn_res)
        


# Create folder with resized images

In [ ]:
from sys import platform
import PIL.Image
import os
from PIL import Image

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining the aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return im, new_im # Return both the cropped and square images

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to get PNG files from a folder that have corresponding mask files
def get_pngs_without_masks(f):
    png_names = get_all_png_from_folder(f)
    return [x for x in png_names if not x.endswith('mask.png') and x[:-4] + '_mask.png' in png_names]

main_path = 'D:/'
# Define the paths for the images with masks and the results folder
with_masks_path = main_path + 'Datasets/create_masks/'
res_folder = main_path + 'Datasets/without_masks_all_reduced512_uncropped'
main_path += 'Datasets/V1-20230907_130717/images/' 

# Get all PNG images that have corresponding masks
pngs_having_maks = get_pngs_without_masks(with_masks_path)
print(len(pngs_having_maks)) # Print the number of images with masks

# Create the results folder if it does not exist
if not os.path.exists(res_folder):
   os.makedirs(res_folder)

# Loop through each image that has a corresponding mask
for image_name in pngs_having_maks: #get_all_png_from_folder(with_masks_path):    
    image = PIL.Image.open(with_masks_path + image_name)
    cropped_im, image = make_square(image) # Make the image square
    image.save(res_folder + '/' + image_name) # Save the square image in the results folder

    image_mask = PIL.Image.open(with_masks_path + image_name[:-4] + '_mask.png') # Open the corresponding mask image
    cropped_im, image_m = make_square(image_mask) # Make the mask image square
    image_m.save(res_folder + '/' + image_name[:-4] + '_mask.png') # Save the square mask image in the results folder


# Create folders of 10 for each initial image

In [ ]:
import os
import shutil

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to get unique root names (folders) from a list of PNG files
def get_all_unique_folders(all_pngs):
    unique_folders = [png.split('__')[0] for png in all_pngs]
    return list(set(unique_folders)) # Return unique root names

# Function to get all derivative files from a root name
def get_all_derivates_from_root_name(name, all_pngs):
    return [x for x in all_pngs if name in x]

main_path = 'D:/'
# Define the paths for the input and output folders
main_path += 'Experiments/Results/'
input_path = main_path + 'Diffusion-all/'
output_path = main_path + 'Diffusion-folders-of-10/'

# Create the output folder if it does not exist
if not os.path.exists(output_path):
   os.makedirs(output_path)

# Get all PNG files from the input folder
all_pngs = get_all_png_from_folder(input_path)
unique_folders = get_all_unique_folders(all_pngs)


for folder in unique_folders:
    current_folder = output_path + folder + '/' # Define the current folder path
    if not os.path.exists(current_folder):
        os.makedirs(current_folder) # Create the current folder if it does not exist
    # Get all derivative files for the current folder
    current_folder_files = get_all_derivates_from_root_name(folder, all_pngs)
    # Copy each derivative file to the current folder
    for current_png in current_folder_files:
        shutil.copyfile(input_path + current_png, output_path + folder + '/' + current_png)
    # Print a message indicating the folder is done
    print('Done with folder', output_path + folder)

# Copy the files that do not have any broken side

In [ ]:
import os
import PIL.Image
from PIL import Image

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to get PNG files from a folder that do not require masks
def get_pngs_without_need_for_masks(f):
    png_names = get_all_png_from_folder(f)
    # Filter out files that are not masks and do not have corresponding mask files
    without_masks = [x for x in png_names if not x.endswith('_mask.png')] 
    return [x for x in without_masks if not x[:-4] + '_mask.png' in png_names]


# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining the aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return im, new_im # Return both the cropped and square images

main_path = 'D:/'
# Define the paths for the input and output folders
input_path = main_path + 'Datasets/create_masks/'
output_path = main_path + 'Experiments/Lama/'
# Get all PNG images that do not need masks
pngs_not_needing_masks = get_pngs_without_need_for_masks(input_path)

# Loop through each PNG file that does not need a mask
for current_png in pngs_not_needing_masks:
    image = PIL.Image.open(input_path + current_png)
    cropped_im, image = make_square(image) # Make the image square
    image.save(output_path + current_png) # Save the square image in the output folder
    print('Done with', output_path + current_png)

# Create a folder with images without black background and only 512 pixels in width

In [33]:
import os
import cv2

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to crop images to 512x512 pixels with a background
def crop_images_512_x_512_with_background(image):
    x = 1920 # Original width
    y = 1080 # Original height
    min_size = 512 # Target size
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining aspect ratio
    w = min_size # Target width
    h = size_y # Target height
    new_x = 0 # X-coordinate for cropping
    new_y = int((min_size - size_y) / 2) # Y-coordinate for cropping
    return image[new_y:new_y+h, new_x:new_x+w] # Crop the image
    
main_path = 'D:/'

# Define the paths for the input and output folders
input_path = main_path + 'Experiments/Diffusion-selected/'
output_path = main_path + 'Experiments/Diffusion-selected-no-background/'

# Create the output folder if it does not exist
if not os.path.exists(output_path):
   os.makedirs(output_path)

# Get all PNG files from the input folder
all_pngs = get_all_png_from_folder(input_path)

# Loop through each PNG file
for current_png in all_pngs:
    image = cv2.imread(input_path + current_png)
    cropped_im = crop_images_512_x_512_with_background(image)
    cv2.imwrite(output_path + current_png, cropped_im)


# Rerun diffusion for one single file

## Redo the mask

In [ ]:
import os
import cv2
import numpy as np

folder = 'D:/Datasets/create_masks/'

# Function to get all PNG files from the folder, excluding those ending with "mask.png"
def get_pngs_from_folder_except_masks(f):
    all_images = []
    for images in os.listdir(f):
        # Check if the image ends with .png and does not end with "mask.png"
        if (images.endswith(".png")) and (not images.endswith("mask.png")):
            all_images.append(images)
    return all_images

# Function to get all PNG files from the folder that have corresponding mask images
def get_pngs_from_folder_having_masks(f):
    all_images_having_masks = []
    all_pngs = os.listdir(f)
    for images in all_pngs:
    
        # Check if the image ends with .png and has a corresponding mask image
        if (images.endswith(".png")) and (images[:-4] + "_mask.png" in all_pngs):
            all_images_having_masks.append(images)
    return all_images_having_masks

# Function to draw a circle on the image and mask when the left mouse button is clicked
def draw_circle(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN: 
        # Draw a green circle on the image
        cv2.circle(img, (x, y), 30, (0, 255, 0), -1) 
        # Draw a white circle on the mask
        cv2.circle(mask, (x, y), 30, 255, -1)
        # Save the mask image     
        cv2.imwrite(folder + file_name[:-4] + '_mask.png', mask)

# Define the window name
window_name = "Redo one file"

# Define the image file to be processed
id_png = 0

file_name = 'frame_00198.png'
print(folder + file_name)
img = cv2.imread(folder + file_name)
# Initialize a mask with zeros (black image)
mask = np.zeros(img.shape[:2], dtype="uint8")


cv2.namedWindow(winname = window_name)
# Set the mouse callback function to 'draw_circle'
cv2.setMouseCallback(window_name, draw_circle) 

# Loop to display the image and handle user inputs
while True:
    cv2.imshow(window_name, img) # Show the current image

    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(10) & 0xFF == 27:
        break
cv2.destroyAllWindows()


# Fill the mask

In [7]:
# Read the partial mask image in grayscale
partial_mask_image = cv2.imread(folder + file_name[:-4] + '_mask.png', cv2.IMREAD_GRAYSCALE)

# Find the contours of the partial mask
contours, _ = cv2.findContours(partial_mask_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty image to draw the filled mask
filled_mask = np.zeros_like(partial_mask_image)

# Draw the contours on the empty image to create the filled mask
cv2.drawContours(filled_mask, contours, -1, 255, thickness=cv2.FILLED)

# Save the filled mask image
output_filled_mask_path = folder + file_name[:-4] + '_mask.png'
cv2.imwrite(output_filled_mask_path, filled_mask)

True

## Resize the mask

In [8]:
import PIL.Image
from PIL import Image

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining the aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return im, new_im # Return both the cropped and square images

main_path = 'D:/'
# Define the paths for the input images and the output folder
with_masks_path = main_path + 'Datasets/create_masks/'
res_folder = main_path + 'Datasets/without_masks_all_reduced512_uncropped'


image = PIL.Image.open(with_masks_path + file_name)

# Make the image square and save it
cropped_im, image = make_square(image)
image.save(res_folder + '/' + file_name)

# Open the corresponding mask image
image_mask = PIL.Image.open(with_masks_path + file_name[:-4] + '_mask.png')

# Make the mask image square and save it
cropped_im, image_m = make_square(image_mask)
image_m.save(res_folder + '/' + file_name[:-4] + '_mask.png')


## Rerun difussion

In [ ]:
import PIL.Image
import os

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to get PNG files from a folder that do not have corresponding mask files
def get_pngs_without_masks(f):
    png_names = get_all_png_from_folder(f)
    return [x for x in png_names if not x.endswith('mask.png')]

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return new_im # Return the new square image

# Function to resize an image to 512x512 pixels
def resize_to_512(img):
    w, h = img.size
    if w != 512 or h != 512:
        return img.resize((512, 512), PIL.Image.Resampling.LANCZOS)
    return img

# Function to read an image, its mask, and a masked colored image from given folders
def read_one_file(full_path_separate_folder, full_path_colored_folder, current_folder, fn):
    mask_name = fn[:-4] + '_mask.png'
    image = PIL.Image.open(full_path_separate_folder + current_folder + '/' + fn)
    image = resize_to_512(image)


    mask_image = PIL.Image.open(full_path_separate_folder + current_folder + '/' + mask_name)
    mask_image = resize_to_512(mask_image)

    masked_colored_image = PIL.Image.open(full_path_colored_folder + current_folder + '/' + mask_name)
    masked_colored_image = resize_to_512(masked_colored_image)

    return image, mask_image, masked_colored_image

# Function to get paths of the separate and masked folders, and list of folders within the main path
def get_paths(main_path):
    separate_folder = 'Separate_files/'
    masked_folder = 'Mask_on_image/'

    full_path_separate_folder = main_path + separate_folder
    full_path_colored_folder = main_path + masked_folder

    return full_path_separate_folder, full_path_colored_folder, os.listdir(full_path_separate_folder)

# Function to get PNG files from a folder that do not have corresponding mask files
def get_png_without_masks(path_to_folder):
    all_files_in_folder = os.listdir(path_to_folder)
    return [x for x in all_files_in_folder if 'mask' not in x and x.endswith('.png')]

# Function to get results from a pipeline (inpainting)
def get_pipe_results(image, prompt = "background", guidance_scale=7.5, num_samples = 10):
    generator = torch.Generator(device="cuda").manual_seed(0) # change the seed to get different results
    
    import time

    start = time.time()

    # Disable the safety checker of the pipeline
    pipe.safety_checker = lambda images, clip_input: (images, False)
    images = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask_image,
        guidance_scale=guidance_scale,
        generator=generator,
        num_images_per_prompt=num_samples,
    ).images
    
    return images, time.time() - start # Return the generated images and the time taken

main_path = 'D:/'
# Define the paths for the results folder
res_folder = main_path + 'Experiments/Diffusion-folders-of-10/' + file_name[:-4] + '/'
main_path += 'Datasets/without_masks_all_reduced512_uncropped/'

# Create the results folder if it does not exist
if not os.path.exists(res_folder):
   os.makedirs(res_folder)



image = PIL.Image.open(main_path + file_name)

# Open the corresponding mask image
mask_image = PIL.Image.open(main_path + file_name[:-4] + '_mask.png')
# Get the results from the pipeline
resulted_images, seconds = get_pipe_results(image)

# Save each generated image
for counter, res_img in enumerate(resulted_images):
    fn_res = file_name[:-4] + '__v_' + str(counter) + '__sec_' + '{:.2f}'.format(seconds / 10) + '.png'
    res_img.save(res_folder + '/' + fn_res)
        


# Run diffusion for checking accuracy

## Resize images

In [13]:
import PIL.Image
from PIL import Image
import os

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return new_im # Return the new square image

main_path = 'D:/'

# Define the paths for the input and output folders
input_path = main_path + 'Datasets/masks_for_inpainting_check/'
all_folders = os.listdir(input_path)
output_path = main_path + 'Datasets/masks_for_inpainting_check_resized/'

# Loop through each folder in the input path
for folder in all_folders:
    print('Processing ' + folder + '...')
    current_folder = input_path + folder + '/'
    current_output_folder = output_path + folder + '/'
    # Create the output folder if it does not exist
    if not os.path.exists(current_output_folder):
        os.makedirs(current_output_folder)
    # Get all PNG files from the current folder
    all_files = get_all_png_from_folder(current_folder)
    # Loop through each PNG file
    for fn in all_files:
        image = PIL.Image.open(current_folder + fn)
        new_image = make_square(image)
        new_image.save(current_output_folder + fn)



Processing Large-margins...
Processing Margins...
Processing Margins-and-center...
Processing Rectangle...


In [ ]:
import os
from sys import platform
import PIL.Image

# Function to get all PNG files from a folder
def get_all_png_from_folder(f):
    all_images = []
    for images in os.listdir(f):
    
        # check if the image ends with png
        if (images.endswith(".png")):
            all_images.append(images)
    return all_images

# Function to resize an image to a square of a specified size, padding with a fill color (black)
def make_square(im, min_size=512, fill_color=(0, 0, 0, 0)):
    x, y = im.size # Get the original dimensions of the image
    size_y = (int)(y * min_size / x) # Calculate the new height while maintaining aspect ratio
    im = im.resize((min_size, size_y), PIL.Image.Resampling.LANCZOS) # Resize the image using LANCZOS resampling
    new_im = Image.new('RGBA', (min_size, min_size), fill_color) # Create a new square image with the specified fill color
    new_im.paste(im, (0, int((min_size - size_y) / 2))) # Paste the resized image in the center of the new square image
    return new_im # Return the new square image

# Function to get PNG files from a folder that do not have corresponding mask files
def get_pngs_without_masks(f):
    png_names = get_all_png_from_folder(f)
    return [x for x in png_names if not x.endswith('mask.png')]

# Function to get results from a pipeline (inpainting)
def get_pipe_results(image, prompt = "background", guidance_scale=7.5, num_samples = 10):
    generator = torch.Generator(device="cuda").manual_seed(0) # change the seed to get different results
    
    import time

    start = time.time()

    #Before adding the line below, I often got:
    # "Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed."
    pipe.safety_checker = lambda images, clip_input: (images, False)
    images = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask_image,
        guidance_scale=guidance_scale,
        generator=generator,
        num_images_per_prompt=num_samples,
    ).images
    
    return images, time.time() - start

main_path = 'D:/'
# Define the paths for the input and output folders
input_path = main_path + 'Datasets/masks_for_inpainting_check_resized/'
all_folders = os.listdir(input_path)
res_folder = main_path + 'Experiments/Diffusion-check/'

# Loop through each folder in the input path
for folder in all_folders:
    print('Processing ' + folder + '...')
    current_folder = input_path + folder + '/'
    current_output_folder = res_folder + folder + '/'
    # Get PNG files from the current folder that do not have corresponding mask files
    all_files = get_pngs_without_masks(current_folder)
    # Loop through each PNG file
    for fn in all_files:
        current_output_fn_folder = current_output_folder + fn[:-4] + '/'
         # Create the output folder for the current file if it does not exist
        if not os.path.exists(current_output_fn_folder):
            os.makedirs(current_output_fn_folder)
        image = PIL.Image.open(current_folder + fn)
        mask_image = PIL.Image.open(current_folder + fn[:-4] + '_mask.png')
        resulted_images, seconds = get_pipe_results(image)
        # Save each generated image
        for counter, res_img in enumerate(resulted_images):
            fn_res = fn[:-4] + '__v_' + str(counter) + '__sec_' + '{:.2f}'.format(seconds / 10) + '.png'
            res_img.save(current_output_fn_folder + '/' + fn_res)

# Run diffusion for one image

In [ ]:
from PIL import Image
import PIL.Image

# Function to get results from a pipeline (inpainting)
def get_pipe_results(image, prompt = "background", guidance_scale=7.5, num_samples = 10):
    generator = torch.Generator(device="cuda").manual_seed(0) # change the seed to get different results
    
    import time

    start = time.time()

    # Disable the safety checker of the pipeline
    pipe.safety_checker = lambda images, clip_input: (images, False)
    images = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask_image,
        guidance_scale=guidance_scale,
        generator=generator,
        num_images_per_prompt=num_samples,
    ).images
    
    return images, time.time() - start

res_folder = 'D:/code/inpainting/Diffusion/'

# Open the input image
image = PIL.Image.open('D:/code/inpainting/input/frame_00098.png')

# Open the corresponding mask image
mask_image = PIL.Image.open('D:/code/inpainting/input/frame_00098_mask.png')

resulted_images, seconds = get_pipe_results(image)

# Loop through each generated image
for counter, res_img in enumerate(resulted_images):
    # Define the filename for the result image
    fn_res = 'frame_00098__v_' + str(counter) + '__sec_' + '{:.2f}'.format(seconds / 10) + '.png'
    # Save the result image to the results folder
    res_img.save(res_folder + '/' + fn_res)